In [2]:
import mysql.connector
import arcturis as a

# Connect to local MySQL database
local_db = mysql.connector.connect(
    host=a.locale,
    user=a.localu,
    password=a.localp,
    database=a.dbe
)

# Connect to cloud MySQL database
cloud_db = mysql.connector.connect(
    host=a.line,
    user=a.linu,
    password=a.linp,
    database=a.lindb
)

# Define the table name
table_name = 'clinical_trials_test'

# Create cursors for both databases
local_cursor = local_db.cursor()
cloud_cursor = cloud_db.cursor()

# Check if the table exists in the cloud database
cloud_cursor.execute("SHOW TABLES LIKE %s", (table_name,))
table_exists = cloud_cursor.fetchone()

# If the table doesn't exist, create it
if not table_exists:
    # Retrieve table structure from local database
    local_cursor.execute(f"SHOW CREATE TABLE {table_name}")
    create_table_query = local_cursor.fetchone()[1]
    
    # Execute the create table query in the cloud database
    cloud_cursor.execute(create_table_query)

# Retrieve data from the local table
local_cursor.execute(f"SELECT * FROM {table_name}")
rows = local_cursor.fetchall()

# Insert data into the cloud table
for row in rows:
    cloud_cursor.execute(f"INSERT INTO {table_name} VALUES ({', '.join(['%s'] * len(row))})", row)

# Commit changes and close connections
cloud_db.commit()
local_cursor.close()
cloud_cursor.close()
local_db.close()
cloud_db.close()



OperationalError: 2013 (HY000): Lost connection to MySQL server during query